<a href="https://colab.research.google.com/github/Qwer1ty7/Lab2/blob/main/2lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imap_tools
!pip install cryptography

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

def generateKey(public_exponent : int = 65537, key_size : int = 4096):
        private_key = rsa.generate_private_key(
            public_exponent = public_exponent,
            key_size = key_size,
            backend = default_backend(),
        )

        with open('private.key', 'wb') as f:
            f.write(
                private_key.private_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PrivateFormat.TraditionalOpenSSL,
                    encryption_algorithm=serialization.NoEncryption(),
                )
            )

        with open('public.pem', 'wb') as f:
            f.write(
                private_key.public_key().public_bytes(
                    encoding = serialization.Encoding.PEM,
                    format = serialization.PublicFormat.SubjectPublicKeyInfo,
                )
            )

In [ ]:
from email.header import decode_header
import os
from imap_tools import MailBox

def recieve(user, password):

    with MailBox('imap.gmail.com').login(user, password, 'INBOX') as mailbox:
            recievedFiles = []
            sentFileName = ''
            msg = next(mailbox.fetch(mark_seen = False, reverse = True))
            for att in msg.attachments:
                with open(os.path.join('recieved', att.filename), 'wb') as f:
                    f.write(att.payload)
                recievedFiles.append(att.filename)
                if att.filename != "public.pem" and att.filename != "signature.sig":
                    sentFileName = att.filename
            if "public.pem" in recievedFiles and\
                "signature.sig" in recievedFiles and\
                len(recievedFiles) == 3:
                return sentFileName
            else:
                return False

In [ ]:
import base64
import cryptography.exceptions
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.serialization import load_pem_public_key
import os

def isSignatureReal(filename):
    with open('recieved/public.pem', 'rb') as f:
        public_key = load_pem_public_key(f.read(), default_backend())

    with open(os.path.join('recieved', filename), 'rb') as f:
        payload_contents = f.read()
    with open('recieved/signature.sig', 'rb') as f:
        signature = base64.b64decode(f.read())

    try:
        public_key.verify(
            signature,
            payload_contents,
            padding.PSS(
                mgf = padding.MGF1(hashes.SHA256()),
                salt_length = padding.PSS.MAX_LENGTH,
            ),
            hashes.SHA256(),
        )
        return True
    except cryptography.exceptions.InvalidSignature as e:
        return False

In [ ]:
import smtplib, ssl
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

def send_mail(send_from, password, send_to, subject, text, files=None,
              smtp_server = "smtp.gmail.com", smtp_port = 587):

    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = COMMASPACE.join(send_to)
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = subject

    msg.attach(MIMEText(text))

    for f in files or []:
        with open(f, "rb") as fil:
            part = MIMEApplication(
                fil.read(),
                Name=basename(f)
            )
        part['Content-Disposition'] = 'attachment; filename="%s"' % basename(f)
        msg.attach(part)

    gmail = send_from

    context = ssl.create_default_context()

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls(context=context)
        server.ehlo()
        server.login(gmail, password)
        server.sendmail(send_from, send_to, msg.as_string())
        server.quit()

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

def generateKey(public_exponent : int = 65537, key_size : int = 4096):
        private_key = rsa.generate_private_key(
            public_exponent = public_exponent,
            key_size = key_size,
            backend = default_backend(),
        )

        with open('private.key', 'wb') as f:
            f.write(
                private_key.private_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PrivateFormat.TraditionalOpenSSL,
                    encryption_algorithm=serialization.NoEncryption(),
                )
            )

        with open('public.pem', 'wb') as f:
            f.write(
                private_key.public_key().public_bytes(
                    encoding = serialization.Encoding.PEM,
                    format = serialization.PublicFormat.SubjectPublicKeyInfo,
                )
            )

In [ ]:
import base64
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding

def sign(filename : str):

    generateKey()

    # Load the private key. 
    with open('private.key', 'rb') as key_file: 
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password = None,
            backend = default_backend(),
        )

    with open(filename, 'rb') as f:
        payload = f.read()

    signature = base64.b64encode(
        private_key.sign(
            payload,
            padding.PSS(
                mgf = padding.MGF1(hashes.SHA256()),
                salt_length = padding.PSS.MAX_LENGTH,
            ),
            hashes.SHA256(),
        )
    )
    with open('signature.sig', 'wb') as f:
        f.write(signature)

In [ ]:
import base64
import cryptography.exceptions
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.serialization import load_pem_public_key
import os

def isSignatureReal(filename):
    with open('recieved/public.pem', 'rb') as f:
        public_key = load_pem_public_key(f.read(), default_backend())

    with open(os.path.join('recieved', filename), 'rb') as f:
        payload_contents = f.read()
    with open('recieved/signature.sig', 'rb') as f:
        signature = base64.b64decode(f.read())

    try:
        public_key.verify(
            signature,
            payload_contents,
            padding.PSS(
                mgf = padding.MGF1(hashes.SHA256()),
                salt_length = padding.PSS.MAX_LENGTH,
            ),
            hashes.SHA256(),
        )
        return True
    except cryptography.exceptions.InvalidSignature as e:
        return False

In [ ]:
import getpass

email_from = "sender228uw@gmail.com"
email_to_list = ["reciever228uw@gmail.com"]
subject = "Send file"
text = "File" 
file = "kek.jpg"
additional_files = ["public.pem", "signature.sig"]
attachments = additional_files + [file]

password = getpass.getpass()

sign(file)
send_mail(email_from, password, email_to_list, subject, text, attachments)

··········


In [ ]:
email_user = input('Email: ')
email_pass = input('Password: ')

filename = recieve(email_user, email_pass)
if filename:
    print(isSignatureReal(filename))
else:
    print('An error occured')